In [ ]:
from hydromt_sfincs import SfincsModel
from os.path import join
import xarray as xr
import numpy as np

In [ ]:
# FIXME
mod0 = SfincsModel('../02_models/beira/sfincs/03_11_htot_dis_precip', mode='r', deltares_data=True)
mod1 = SfincsModel('../02_models/beira/sfincs/03_17_htide_disclim_precip', mode='r', deltares_data=True)
trange = slice('20210118T0000','20210128T0000')
event = 'eloise'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
forcing = mod0.forcing
kwargs=dict()

colors = ['#377eb8', '#ff7f00', '#4daf4a', '#f781bf', '#a65628', '#984ea3', '#999999', '#e41a1c', '#dede00']

n = len(forcing.keys())
kwargs0 = dict(sharex=True, figsize=(9, n * 3))
kwargs0.update(**kwargs)
fig, axes = plt.subplots(n, 1, **kwargs0)
axes = [axes] if n == 1 else axes
for i, name in enumerate(forcing):
    da = forcing[name].sel(time=trange)
    longname = da.attrs.get("standard_name", "")
    unit = da.attrs.get("unit", "")
    prefix = ""
    if da.ndim == 3:
        da = da.mean(dim=[da.raster.x_dim, da.raster.y_dim])
        prefix = "mean "
    # convert to Single index dataframe (bar plots don't work with xarray)
    df = da.squeeze().to_series()
    if isinstance(df.index, pd.MultiIndex):
        df = df.unstack(0)
    # convert dates a-priori as automatic conversion doesn't always work
    df.index = mdates.date2num(df.index)
    if longname == "precipitation":
        axes[i].bar(df.index, df.values, facecolor="darkblue")
        longname = 'runoff'
    else:
        # tide / disclim
        da1 = mod1.forcing[name].sel(time=trange)
        df1 = da1.squeeze().to_series().unstack(0)
        df1.index = mdates.date2num(df1.index)
        
        bound = 'H' if longname == 'waterlevel' else 'Q'
        cols = [1] if longname == 'waterlevel' else [0,3]
        ylim = [df.values.min()*1.05, df.values.max()*1.3] if longname == 'waterlevel' else [0, df.values.max()*1.3]
        j0 = 0 if longname == 'waterlevel' else 1
        
        # plot actual
        suffix = 'tot' if longname == 'waterlevel' else 'event'
        df.columns = [f'{bound}{i} - {suffix}' for i in df.columns.values]
        for j, col in enumerate(df.columns[cols]):
            df.loc[:,col].plot.line(ax=axes[i], c=colors[j+j0], label=col)
            
        # plot tide/clim
        suffix = 'tide' if longname == 'waterlevel' else 'clim.'
        df1.columns = [f'{bound}{i} - {suffix}' for i in df1.columns.values]
        for j, col in enumerate(df1.columns[cols]):
            df1.loc[:,col].plot.line(ax=axes[i], ls='--', color=colors[j+j0], label=col)


        axes[i].legend(
#             title="index",
            bbox_to_anchor=(1, 1),
            loc="upper right",
            ncol=2,
        )
        axes[i].set_ylim(ylim)

    axes[i].set_ylabel(f"{prefix}{longname}\n[{unit}]")
    axes[i].set_title(f"{longname} boundary")
    axes[i].set_xlim([df.index[0], df.index[-1]])

# use a concise date formatter for format date axis ticks
locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
axes[-1].xaxis.set_major_locator(locator)
axes[-1].xaxis.set_major_formatter(formatter)

plt.savefig(join('figs', f'forcing_{event}.png'), dpi=225, bbox_inches="tight")